In [4]:
from utils import *
df_tracks = read_avro('data/tracks.avro')

In [122]:
from utils import *
df_artists = read_avro('data/artists.avro')
df_artists.head()

,id,followers,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,Ioannis Panoutsopoulos,0


In [3]:
df_artists = pd.read_csv(
    '/Users/yassine/Desktop/IE/4th year/2nd sem/stream analytics/datasets/artists.csv')
df_artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15


In [5]:
import pandas as pd
import numpy as np
# Because release_date is a string, and comes in different formats , we will first tackle the YYYY format , then YYYY-MM then YYYY-MM-DD 
mask = df_tracks['release_date'].astype(str).apply(len) == 4

df_tracks.loc[mask, 'months_elapsed'] = (2024 - pd.to_numeric(
    df_tracks.loc[mask, 'release_date'])+1) * 12  # +1 to include the release year

reference_date = pd.to_datetime('2024-01-01')

try:
    months_elapsed, _ = divmod((reference_date - pd.to_datetime(
        df_tracks.loc[~mask, 'release_date'], errors='coerce')).dt.days, 30)
    df_tracks.loc[~mask, 'months_elapsed'] = months_elapsed.fillna(0)
except ValueError as e:
    print(f"Error: {e}")
    print("Problematic values:")
    print(df_tracks.loc[~mask, 'release_date'])
    df_tracks.loc[~mask, 'months_elapsed'] = np.nan

print(df_tracks[['release_date', 'months_elapsed']])

       release_date  months_elapsed
0        1922-02-22          1240.0
1        1922-06-01          1236.0
2        1922-03-21          1239.0
3        1922-03-21          1239.0
4              1922          1236.0
...             ...             ...
586596   2020-09-26            39.0
586597   2020-10-21            38.0
586598   2020-09-02            40.0
586599   2021-03-05            34.0
586600   2015-07-01           103.0

[586601 rows x 2 columns]


In [108]:
# Assuming df_tracks is your DataFrame
min_release_date = df_tracks['release_date'].min()
max_release_date = df_tracks['release_date'].max()

min_months_elapsed = df_tracks['months_elapsed'].min()
max_months_elapsed = df_tracks['months_elapsed'].max()

print("Min Release Date:", min_release_date)
print("Max Release Date:", max_release_date)

print("Min Months Elapsed:", min_months_elapsed)
print("Max Months Elapsed:", max_months_elapsed)


Min Release Date: 1900-01-01
Max Release Date: 2021-04-16
Min Months Elapsed: 0.0
Max Months Elapsed: 1509.0


In [110]:
min_months_elapsed = df_tracks['months_elapsed'].min()
max_months_elapsed = df_tracks['months_elapsed'].max()

# Normalize 'months_elapsed' to the range [0, 1]
df_tracks['newness_score'] = 1 - (df_tracks['months_elapsed'] - min_months_elapsed) / (max_months_elapsed - min_months_elapsed)
df_tracks['newness_score'] = df_tracks['newness_score'].clip(0, 1)

print(df_tracks[['release_date', 'newness_score']])


       release_date  newness_score
0        1922-02-22       0.178264
1        1922-06-01       0.180915
2        1922-03-21       0.178926
3        1922-03-21       0.178926
4              1922       0.180915
...             ...            ...
586596   2020-09-26       0.974155
586597   2020-10-21       0.974818
586598   2020-09-02       0.973492
586599   2021-03-05       0.977469
586600   2015-07-01       0.931743

[586601 rows x 2 columns]


In [111]:
df_tracks.columns

Index(['track_id', 'duration', 'artist', 'name', 'popularity', 'release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'months_elapsed', 'newness_score', 'genre'],
      dtype='object')

In [112]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Extract relevant audio features for clustering
audio_features = df_tracks[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

# Standardize the data
scaler = StandardScaler()
scaled_audio_features = scaler.fit_transform(audio_features)

num_clusters = 10  

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_tracks['genre'] = kmeans.fit_predict(scaled_audio_features)

print(df_tracks[['track_id', 'genre']])

/Users/yassine/miniconda3/envs/stream_analytics_project/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                      track_id  genre
0       35iwgR4jXetI318WEWsa1Q      3
1       021ht4sdgPcrDgSk7JTbKY      7
2       07A5yehtSnoedViJAZkNnc      1
3       08FmqUhxtyLTn6pAh6bk45      6
4       08y9GfoqCWfOGsKdwojr5e      8
...                        ...    ...
586596  5rgu12WBIHQtvej2MdHSH0      8
586597  0NuWgxEp51CutD2pJoF4OM      5
586598  27Y1N4Q4U3EfDU5Ubw8ws2      8
586599  45XJsGpFTyzbzeWK8VzR8S      9
586600  5Ocn6dZ3BJFPWh4ylwFXtn      9

[586601 rows x 2 columns]


In [113]:
selected_columns = ['track_id', 'duration', 'artist', 'name', 'popularity', 'newness_score', 'genre']
df_final = df_tracks[selected_columns]

df_final

,track_id,duration,artist,name,popularity,newness_score,genre
0,35iwgR4jXetI318WEWsa1Q,126903,['Uli'],Carve,6,0.178264,3
1,021ht4sdgPcrDgSk7JTbKY,98200,['Fernando Pessoa'],Capítulo 2.16 - Banquero Anarquista,0,0.180915,7
2,07A5yehtSnoedViJAZkNnc,181640,['Ignacio Corsini'],Vivo para Quererte - Remasterizado,0,0.178926,1
3,08FmqUhxtyLTn6pAh6bk45,176907,['Ignacio Corsini'],El Prisionero - Remasterizado,0,0.178926,6
4,08y9GfoqCWfOGsKdwojr5e,163080,['Dick Haymes'],Lady of the Evening,0,0.180915,8
...,...,...,...,...,...,...,...
586596,5rgu12WBIHQtvej2MdHSH0,258267,['阿YueYue'],云与海,50,0.974155,8
586597,0NuWgxEp51CutD2pJoF4OM,153293,['ROLE MODEL'],blind,72,0.974818,5
586598,27Y1N4Q4U3EfDU5Ubw8ws2,187601,['FINNEAS'],What They'll Say About Us,70,0.973492,8
586599,45XJsGpFTyzbzeWK8VzR8S,142003,"['Gentle Bones', 'Clara Benin']",A Day At A Time,58,0.977469,9


In [6]:
df_tracks.name

0                                       Carve
1         Capítulo 2.16 - Banquero Anarquista
2          Vivo para Quererte - Remasterizado
3               El Prisionero - Remasterizado
4                         Lady of the Evening
                         ...                 
586596                                    云与海
586597                                  blind
586598              What They'll Say About Us
586599                        A Day At A Time
586600                       Mar de Emociones
Name: name, Length: 586601, dtype: object

In [22]:
df_2 = pd.read_csv('/Users/yassine/Desktop/IE/4th year/2nd sem/stream analytics/datasets/dataset.csv')
df_2.head(10)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,...,-8.807,1,0.1050,0.2890,0.000000,0.1890,0.6660,98.017,4,acoustic
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,...,-8.822,1,0.0355,0.8570,0.000003,0.0913,0.0765,141.284,3,acoustic
7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,...,-9.331,1,0.0417,0.5590,0.000000,0.0973,0.7120,150.960,4,acoustic
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,...,-8.700,1,0.0369,0.2940,0.000000,0.1510,0.6690,130.088,4,acoustic
9,9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.6320,...,-6.770,1,0.0295,0.4260,0.004190,0.0735,0.1960,78.899,4,acoustic


In [23]:
df_2.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [98]:
print(len(common_songs))

26111


In [99]:
print(len(df_merged))

255105


In [123]:
df_merged.columns

Index(['track_id_x', 'duration', 'artist', 'name', 'popularity_x',
       'release_date', 'danceability_x', 'energy_x', 'key_x', 'loudness_x',
       'mode_x', 'speechiness_x', 'acousticness_x', 'instrumentalness_x',
       'liveness_x', 'valence_x', 'tempo_x', 'time_signature_x',
       'months_elapsed', 'Unnamed: 0', 'track_id_y', 'artists', 'album_name',
       'track_name', 'popularity_y', 'duration_ms', 'explicit',
       'danceability_y', 'energy_y', 'key_y', 'loudness_y', 'mode_y',
       'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
       'valence_y', 'tempo_y', 'time_signature_y', 'track_genre'],
      dtype='object')

In [129]:
df_tracks = df_tracks.drop_duplicates(subset='name', keep='first')
df_2 = df_2.drop_duplicates(subset='track_name', keep='first')

common_songs = set(df_tracks['name']).intersection(df_2['track_name'])

df_common_tracks = df_tracks[df_tracks['name'].isin(common_songs)]
df_common_2 = df_2[df_2['track_name'].isin(common_songs)]

df_2_selected = df_common_2[['track_name', 'album_name', 'track_genre']]

df_merged = pd.merge(df_common_tracks, df_2_selected,
                     left_on='name', right_on='track_name', how='inner')

print(len(df_merged))

26112


In [130]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26112 entries, 0 to 26111
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                26112 non-null  object 
 1   name              26111 non-null  object 
 2   popularity        26112 non-null  int64  
 3   duration_ms       26112 non-null  int64  
 4   explicit          26112 non-null  int64  
 5   artists           26112 non-null  object 
 6   id_artists        26112 non-null  object 
 7   release_date      26112 non-null  object 
 8   danceability      26112 non-null  float64
 9   energy            26112 non-null  float64
 10  key               26112 non-null  int64  
 11  loudness          26112 non-null  float64
 12  mode              26112 non-null  int64  
 13  speechiness       26112 non-null  float64
 14  acousticness      26112 non-null  float64
 15  instrumentalness  26112 non-null  float64
 16  liveness          26112 non-null  float6

In [131]:
print(df_merged['artist'].dtype)


KeyError: 'artist'

In [128]:
print(len(common_songs))

26112


In [114]:
df_merged['artist'] = df_merged['artist'].astype(str).str.strip("[]").str.replace("'", "")


In [71]:
# Assuming 'name' is the column in df_tracks and 'track_name' is the column in df_2
common_songs = set(df_tracks['name']).intersection(df_2['track_name'])

# Count of common songs
num_common_songs = len(common_songs)

# Print or use the count as needed
print("Number of common songs:", num_common_songs)


Number of common songs: 26111


In [119]:
df_artists = pd.read_csv('/Users/yassine/Desktop/IE/4th year/2nd sem/stream analytics/datasets/artists.csv')


In [73]:
df_merged.artist.unique()

array(['Dick Haymes', 'Beniamino Gigli', 'Carlo Buti', ...,
       'Ludwig van Beethoven, Igor Levit', 'Listener Kids', 'KIMSEJEONG'],
      dtype=object)

In [74]:
df_artists.name

0          Armid & Amir Zare Pashai feat. Sara Rouzbehani
1                                             ปูนา ภาวิณี
2                                                   Sadaa
3                                               Tra'gruda
4                                  Ioannis Panoutsopoulos
                                ...                      
1162090                                        Ali Siddiq
1162091                                      Rodney Laney
1162092                                      Blake Wexler
1162093                                  Donnell Rawlings
1162094                                          Gabe Kea
Name: name, Length: 1162095, dtype: object

In [85]:
# Assuming 'name_set' is the column in df_artists and 'artist_set' is the column in df_merged
common_artists_df = pd.merge(df_merged, df_artists, left_on='artist', right_on='name', how='inner')

# Print or use common_artists_df as needed
print(len(common_artists_df))


238102


In [88]:
# Unique artist names in df_merged
unique_artists_merged = set(df_merged['artist'].apply(lambda x: x[0] if isinstance(x, list) else x))

# Unique artist names in df_artists
unique_artists_artists = set(df_artists['name'].apply(lambda x: x[0] if isinstance(x, list) else x))

# Find the common artists using sets
common_artists_set = unique_artists_merged.intersection(unique_artists_artists)

# Print the counts and unique artists
print(f"Count of common artists: {len(common_artists_set)}")


Count of common artists: 16827
Unique artists: {'Chapa C', 'Mente Fuerte', 'Rene Liu', 'Noizu', 'Jean Musy', 'Nani Azevedo', 'Rick & Renner', 'Motel Connection', 'Throttle', 'Luiz Henrique', 'Blind Faith', 'Little Richard', 'Kelvin Jones', 'Golpe De Estado', 'Porca Véia', 'Lester Young, Oscar Peterson Trio', 'Congreso', 'Heroes Del Silencio', 'Emancipator', 'mike.', 'Demarco', 'Mink DeVille', 'Be Peerapat', 'The Smithereens', 'Midian Lima', 'Jósean Log', 'La Fuga', 'LeAnn Rimes', 'The Woolies', 'Pat Bringer', 'The Mothers Of Invention', 'Little Joe And The Latinaires', 'Elefánt', 'Lori Spee', 'Refuzion', 'Antonio Tarrago Ros', 'Dark Hills Gang', 'Lasso', 'The Assembly', 'Los Askis', '郭金發', 'Alex Porat', 'Agnes Chiang', 'Compay Quinto', 'Momoe Yamaguchi', 'Larry Williams', 'Amitabh Bachchan', 'Xiii. Stoleti', 'SAC1', 'Thomas Jack', 'Sheryl Crow', 'Cheryl Mansueto', 'Kadri Voorand', 'Amel Bent', 'The Edgar Winter Group', 'Fiona Sit', 'Itchy Daze', 'Herman Brood', 'Mark Wills', 'Jennifer 

In [90]:
# Find the common artists using sets
common_artists_set = set(df_merged['artist']).intersection(df_artists['name'])

# Print the count and common artists
print(f"Count of common artists: {len(common_artists_set)}")


Count of common artists: 16827


In [116]:
import pandas as pd

# Find the common artists using sets
common_artists_set = set(df_merged['artist']).intersection(df_artists['name'])

# Create a DataFrame with the common artists
df_common_artists = pd.DataFrame(list(common_artists_set), columns=['common_artist'])

# Print the count and the DataFrame
print(f"Count of common artists: {len(common_artists_set)}")



Count of common artists: 16827


In [113]:
df_merged

,track_id_x,duration,artist,name,popularity_x,release_date,danceability_x,energy_x,key_x,loudness_x,...,loudness_y,mode_y,speechiness_y,acousticness_y,instrumentalness_y,liveness_y,valence_y,tempo_y,time_signature_y,track_genre
0,0BRXJHRNGQ3W4v9frnSfhu,178933,['Dick Haymes'],Ave Maria,0,1922,0.227,0.261,5,-12.343,...,-21.942,1,0.0405,0.987,0.894000,0.0687,0.0543,81.807,3,classical
1,0BRXJHRNGQ3W4v9frnSfhu,178933,['Dick Haymes'],Ave Maria,0,1922,0.227,0.261,5,-12.343,...,-21.942,1,0.0405,0.987,0.894000,0.0687,0.0543,81.807,3,classical
2,0BRXJHRNGQ3W4v9frnSfhu,178933,['Dick Haymes'],Ave Maria,0,1922,0.227,0.261,5,-12.343,...,-21.942,1,0.0405,0.987,0.894000,0.0687,0.0543,81.807,3,german
3,0BRXJHRNGQ3W4v9frnSfhu,178933,['Dick Haymes'],Ave Maria,0,1922,0.227,0.261,5,-12.343,...,-21.942,1,0.0405,0.987,0.894000,0.0687,0.0543,81.807,3,german
4,0BRXJHRNGQ3W4v9frnSfhu,178933,['Dick Haymes'],Ave Maria,0,1922,0.227,0.261,5,-12.343,...,-29.760,1,0.0445,0.993,0.888000,0.0569,0.1400,81.616,3,opera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255100,27Y1N4Q4U3EfDU5Ubw8ws2,187601,['FINNEAS'],What They'll Say About Us,70,2020-09-02,0.535,0.314,7,-12.823,...,-13.036,0,0.0417,0.880,0.000073,0.1160,0.0897,144.988,4,electro
255101,27Y1N4Q4U3EfDU5Ubw8ws2,187601,['FINNEAS'],What They'll Say About Us,70,2020-09-02,0.535,0.314,7,-12.823,...,-13.036,0,0.0417,0.880,0.000073,0.1160,0.0897,144.988,4,indie-pop
255102,27Y1N4Q4U3EfDU5Ubw8ws2,187601,['FINNEAS'],What They'll Say About Us,70,2020-09-02,0.535,0.314,7,-12.823,...,-13.036,0,0.0417,0.880,0.000073,0.1160,0.0897,144.988,4,indie-pop
255103,27Y1N4Q4U3EfDU5Ubw8ws2,187601,['FINNEAS'],What They'll Say About Us,70,2020-09-02,0.535,0.314,7,-12.823,...,-13.036,0,0.0417,0.880,0.000073,0.1160,0.0897,144.988,4,indie


In [115]:
# Assuming 'name' is the column in df_tracks and 'track_name' is the column in df_2
common_artists = set(df_merged['artist']).intersection(df_artists['name'])

# Create new datasets with only common songs
df_common_artists = df_merged[df_merged['artist'].isin(common_artists)]
df_common_2 = df_artists[df_artists['name'].isin(common_artists)]

# Perform join on common song names
df_common = pd.merge(df_common_artists, df_common_2, left_on='artist', right_on='name', how='inner')

# Print or use the merged dataframe as needed
print(len(df_common))


238102


In [108]:
df_merged.track_genre.isnull().sum()

0

In [101]:
df_common

,track_id_x,duration,artist,name_x,popularity_x,release_date,danceability_x,energy_x,key_x,loudness_x,...,tempo_y,time_signature_y,track_genre,artist_set,id,followers,genres,name_y,popularity,name_set
0,0VrYjOkQSD8taXLLNlJOeE,168773,Frank Sinatra,Stardust,22,1962,0.240,0.283,10,-14.438,...,149.912,4,dubstep,"{ , i, a, r, k, n, S, t, F}",1Mxqyy3pSjf8kZZL4QVxS0,4677919.0,"['adult standards', 'easy listening', 'lounge']",Frank Sinatra,79,{Frank Sinatra}
1,0VrYjOkQSD8taXLLNlJOeE,168773,Frank Sinatra,Stardust,22,1962,0.240,0.283,10,-14.438,...,109.141,3,jazz,"{ , i, a, r, k, n, S, t, F}",1Mxqyy3pSjf8kZZL4QVxS0,4677919.0,"['adult standards', 'easy listening', 'lounge']",Frank Sinatra,79,{Frank Sinatra}
2,1C0tRiuJgWgFsEUK7XqeQh,183827,Frank Sinatra,Imagination,17,1961-10,0.641,0.235,1,-13.078,...,116.515,4,british,"{ , i, a, r, k, n, S, t, F}",1Mxqyy3pSjf8kZZL4QVxS0,4677919.0,"['adult standards', 'easy listening', 'lounge']",Frank Sinatra,79,{Frank Sinatra}
3,1C0tRiuJgWgFsEUK7XqeQh,183827,Frank Sinatra,Imagination,17,1961-10,0.641,0.235,1,-13.078,...,109.975,5,chill,"{ , i, a, r, k, n, S, t, F}",1Mxqyy3pSjf8kZZL4QVxS0,4677919.0,"['adult standards', 'easy listening', 'lounge']",Frank Sinatra,79,{Frank Sinatra}
4,1C0tRiuJgWgFsEUK7XqeQh,183827,Frank Sinatra,Imagination,17,1961-10,0.641,0.235,1,-13.078,...,115.016,4,chill,"{ , i, a, r, k, n, S, t, F}",1Mxqyy3pSjf8kZZL4QVxS0,4677919.0,"['adult standards', 'easy listening', 'lounge']",Frank Sinatra,79,{Frank Sinatra}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13509,2rCnIKQlqgpcYUEtvnWT6g,228800,Style,Empty Bed,17,1988-10-03,0.627,0.681,2,-15.317,...,123.037,4,club,"{y, S, e, l, t}",5dkIWtbRmQoX1YLilwg4KJ,6795.0,"['classic swedish pop', 'europop', 'swedish pop']",Style,40,{Style}
13510,2rCnIKQlqgpcYUEtvnWT6g,228800,Style,Empty Bed,17,1988-10-03,0.627,0.681,2,-15.317,...,123.037,4,club,"{y, S, e, l, t}",5TXvUwYqvEI4AJB939MN6E,6.0,[],Style,9,{Style}
13511,2rCnIKQlqgpcYUEtvnWT6g,228800,Style,Empty Bed,17,1988-10-03,0.627,0.681,2,-15.317,...,123.037,4,club,"{y, S, e, l, t}",6pKBxxs9jwkuaeXaKEUDLX,4.0,[],Style,8,{Style}
13512,5vJVGPg7qqRDvk4CIenO2T,287533,System,Under Surveillance,0,2014-11-26,0.640,0.726,0,-7.568,...,112.768,4,iranian,"{y, s, S, e, t, m}",4fpGIiXqB2Bhe83iqTCXhB,1008.0,[],System,24,{System}


In [100]:
df_artists.

,id,followers,genres,name,popularity,name_set
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0,{Armid & Amir Zare Pashai feat. Sara Rouzbehani}
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0,{ปูนา ภาวิณี}
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0,{Sadaa}
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0,{Tra'gruda}
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0,{Ioannis Panoutsopoulos}
...,...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34,{Ali Siddiq}
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2,{Rodney Laney}
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10,{Blake Wexler}
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15,{Donnell Rawlings}


In [110]:
df_artists.genres.value_counts()

genres
[]                                                                                                                   856500
['background piano']                                                                                                    307
['calming instrumental']                                                                                                256
['background music']                                                                                                    247
['russelater']                                                                                                          226
                                                                                                                      ...  
['christian hip hop', 'christian pop', 'christian trap', 'miami hip hop', 'rap cristiano']                                1
['edm', 'moldovan pop', 'pop dance', 'progressive house', 'progressive trance', 'trance', 'uplifting trance']             1
[

In [120]:
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162095 entries, 0 to 1162094
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1162095 non-null  object 
 1   followers   1162084 non-null  float64
 2   genres      1162095 non-null  object 
 3   name        1162092 non-null  object 
 4   popularity  1162095 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 44.3+ MB


In [121]:
from fastavro import parse_schema

def get_parsed_artist_schema():
    artist_schema = {
        "doc": "Spotify Wrapped Data Feed - Artist Info",
        "name": "Artist",
        "namespace": "com.spotify.wrapped",
        "type": "record",
        "fields": [
            {"name": "id", "type": "string"},
            {"name": "followers", "type": ["null", "float"], "default": None},
            {"name": "name", "type": ["null", "string"], "default": None},
            {"name": "popularity", "type": "int", "default": 0}
        ]
    }

    return parse_schema(artist_schema)

parsed_artist_schema = get_parsed_artist_schema()
print(parsed_artist_schema)


{'type': 'record', 'doc': 'Spotify Wrapped Data Feed - Artist Info', 'name': 'com.spotify.wrapped.Artist', 'fields': [{'name': 'id', 'type': 'string'}, {'default': None, 'name': 'followers', 'type': ['null', 'float']}, {'default': None, 'name': 'name', 'type': ['null', 'string']}, {'default': 0, 'name': 'popularity', 'type': 'int'}], '__fastavro_parsed': True, '__named_schemas': {'com.spotify.wrapped.Artist': {'type': 'record', 'doc': 'Spotify Wrapped Data Feed - Artist Info', 'name': 'com.spotify.wrapped.Artist', 'fields': [{'name': 'id', 'type': 'string'}, {'default': None, 'name': 'followers', 'type': ['null', 'float']}, {'default': None, 'name': 'name', 'type': ['null', 'string']}, {'default': 0, 'name': 'popularity', 'type': 'int'}]}}}
